In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# tfidf를 설명해보자.
# 자연어는 일상에서 사용하는 언어를 말한다. 사람들이 일상에서 쓰는 언어가 자연어이다. 강아지소리도 자연어는 아니다. 
# 보통 음성인식, 요약, 번역, 감성분석, 텍스트분석, 기사분류, 분류, 질의응답, 챗봇등에 사용된다.
# 자연어처리에 관련된 기술들이 많이 등장하였다. 
# 환경구성 : 주피터노트북, 아나콘다를 설치했을 것이다. 머신러닝,딥러닝,데이터분석(사이킷런) 패키지가 설치되어 있다.
# 추가로 설치해야하는 것은 텐서플로, 젠심, 파이토치를 설치해야하고 한국어는 konlpy를 써야한다. 
# 영어는 nltk가 있지만 주피터노트북에 기본설치가 되어있다. 
# 자연어 처리를 하는 과정을 생각해보자. 
# 1. 텍스트 전처리 절차
# - 토큰화, 정제, 어간추출, 불용어 제거, 정수인코딩(단어를 숫자로 바꾸기), 패딩
# 이러한 작업들을 해야하며 패키지를 이용해야 한다. 
# 2. 텍스트 수치화 표현
# - BoW, DTM/TDM ,TF-IDF등이 사용된다. 
# 다음과 같이 텍스트를 수치로 표현하는 방법이 여러가지가 있다.
# 3. 유사도
# - 문서간의 유사도, 단어간의 유사도, 문장간의 유사도
# 머신러닝/ 딥러닝 알고리즘 사용하여 딥러닝 모델을 생성함
# 먼저 설치해야 하는 것들은 다음과 같다.

In [14]:
pip install konlpy # 한국어 데이터분석 패키지이다. 
# 기본적으로 자연어처리를 위한 여러 도구가 있는데 이중에 한국어패키지이다. 
# 문제는 이것이 자바로 만들어져 있다. 여기는 파이썬이다. 다이렉트로 연결이 잘 안된다. 

Note: you may need to restart the kernel to use updated packages.


In [15]:
from konlpy.tag import Okt #먼저 불러보자. 

In [16]:
okt = Okt()

In [20]:
print(okt.morphs(u'단독입찰보다 복수입찰의 경우')) # 형태소 단위로 나눈 것이다.
print(okt.nouns(u'유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는')) # 명사 추출을 한 것이다.

['단독', '입찰', '보다', '복수', '입찰', '의', '경우']
['항공기', '체계', '종합', '개발', '경험']


In [21]:
# https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0 형태소분석기 이름들이다.
# 여기서 Mecab, Okt을 자주 쓴다. Okt는 원래 네임은 Twitter korean Text이다. 저작권으로 인해서 바꾼 것이다. 
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True)) # 각각 나눠서 이 말의 단어의 속성도 볼 수 있다.
# 이것은 형태소 분석기인데 리스트 안에 튜플로 만들어놓았다. 
# 한나눔, 코코마는 카이스트, 서울대에서 만들었지만 잘 쓰지 않는다. 코모란도 잘 쓰지 않는다. 
# 매켑은 교토대학에서 제작한 것으로 한국어 매켑은 성능이 괜찮다. 
# 유호영님께서 만든 Okt는 성능이 좋기로 유명하다. 

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]


In [22]:
from konlpy.tag import * # 전부 가져와봤다. 물론 이걸 권장하지 않는다. 메모리로 전부 적재되어버리기 때문이다. 

In [24]:
okt = Okt()
han = Hannanum()
kkma = Kkma()
# 먼저 3가지만 해보자. 예를들어 '아버지가방에들어가신다.'가 있다. 띄어쓰기가 안되어 있으면 판단하기 어려울 수 있다.

In [25]:
print(okt.pos('아버지가방에들어가신다.'))
print(han.pos('아버지가방에들어가신다.'))
print(kkma.pos('아버지가방에들어가신다.'))
# 한나눔은 성능이 별로 안좋다는게 보인다. kkma는 의외로 성능이 좋은 편이다. 

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb'), ('.', 'Punctuation')]
[('아버지가방에들어가', 'N'), ('이', 'J'), ('시ㄴ다', 'E'), ('.', 'S')]
[('아버지', 'NNG'), ('가방', 'NNG'), ('에', 'JKM'), ('들어가', 'VV'), ('시', 'EPH'), ('ㄴ다', 'EFN'), ('.', 'SF')]


In [27]:
print(okt.pos("정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."))
print ("\n")
print(han.pos("정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."))
print ("\n")
print(kkma.pos("정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."))
# Punctuation는 콤마같이 기준점이 되는 것들이다. 
# 여기서 내가 원하는 것만 뽑아서 작업을 하면 된다. 

[('정부', 'Noun'), ('가', 'Josa'), ('발표', 'Noun'), ('하는', 'Verb'), ('물가상승률', 'Noun'), ('과', 'Josa'), ('소비자', 'Noun'), ('가', 'Josa'), ('느끼는', 'Verb'), ('물가상승률', 'Noun'), ('은', 'Josa'), ('다르다', 'Adjective'), ('.', 'Punctuation')]


[('정부', 'N'), ('가', 'J'), ('발표', 'N'), ('하', 'X'), ('는', 'E'), ('물가상승률', 'N'), ('과', 'J'), ('소비자', 'N'), ('가', 'J'), ('느끼', 'P'), ('는', 'E'), ('물가상승률', 'N'), ('은', 'J'), ('다르', 'P'), ('다', 'E'), ('.', 'S')]


[('정부', 'NNG'), ('가', 'JKS'), ('발표', 'NNG'), ('하', 'XSV'), ('는', 'ETD'), ('물가', 'NNG'), ('상승률', 'NNG'), ('과', 'JC'), ('소비자', 'NNG'), ('가', 'JKS'), ('느끼', 'VV'), ('는', 'ETD'), ('물가', 'NNG'), ('상승률', 'NNG'), ('은', 'JX'), ('다르', 'VA'), ('다', 'EFN'), ('.', 'SF')]


In [35]:
from math import log 
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]
# 매우 짧은 문서가 4개가 있다. 영화마다 있는 overview랑 비슷하게 생각하자.

In [37]:
vocab=list(set(w for doc in docs for w in doc.split()))# doc을 docs에서 추출해서 스플릿하여 w에 담아라
vocab # 코퍼스(말뭉치) : 전체 문서에 등장하는 단어집합이다. 
# 총 9개의 단어로 구성되어있다. 

['먹고', '좋아요', '싶은', '과일이', '길고', '노란', '사과', '바나나', '저는']

In [38]:
# tfidf의 원리 함수
N = len(docs) # 전체 문서의 길이

def tf(t, d): # 단어와 문서를 전달받는다. 
  return d.count(t) # t라는 단어의 빈도수를 본다. 

def idf(t):
  df = 0
  for doc in docs: 
    df += t in doc # 문서의 내부의 값이 있으면 df에 1씩 더한다. 
  return log(N/(df+1)) # 문서 전체를 로그로 씌운 후 전체문서 길이에 df+1을 나눈다. 

def tfidf(t, d):
  return tf(t,d)* idf(t) # tf와 idf를 곱하는 것이 tfidf이다. 

In [39]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N): # 전체 문서의 길이만큼 반복하라. 
  result.append([])# 결과에 비어있는 리스트를 대입하라.
  d = docs[i] # i번째 문서를 d로 정의하라
  for j in range(len(vocab)): # vocab의 길이만큼 확인하라
    t = vocab[j] # vocap의 단어를 t에 대입해라.
    result[-1].append(tf(t, d)) # 과일의 라는 단어를 tf함수에 전달한다.
                                # tf(원하는 단어, 문서 전체)순으로 정렬하여 문서에 몇번 나왔는지 카운트한다. 
                                # 얘들이 들어가있다. 문서별 단어의 빈도수이다. 이것으로 데이터프레임을 만들었다. 
tf_ = pd.DataFrame(result, columns = vocab)

In [40]:
tf_ # 이것을 tf행렬이라고 부른다. DTM이라고도 부른다. 문서단어행렬이라고도 한다.
# 행에는 문서의 번호가 적혀있다. 열에는 단어집합이 적혀있다.
# 데이터는 빈도수이다. 이것을 문서단어행렬이라고 한다. 
# vocab에 있는 것으로 열이름을 정의했다.

,먹고,좋아요,싶은,과일이,길고,노란,사과,바나나,저는
0,1,0,1,0,0,0,1,0,0
1,1,0,1,0,0,0,0,1,0
2,0,0,0,0,1,1,0,2,0
3,0,1,0,1,0,0,0,0,1


In [42]:
from sklearn.feature_extraction.text import CountVectorizer #이걸 써보자. 카운터 벡터라이저이다. TF를 구하는 목적으로 사용한다. 

In [43]:
vec = CountVectorizer()

In [44]:
vec.fit_transform(docs).toarray() # dtm을 만들때는 이렇게 만들면 된다.  사실상 이게 TF이다. 

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [45]:
vec.vocabulary_ # 열 인덱스 번호이다. 위의 어레이의 인덱스이다.인덱스값이다. 여기까지 TF를 구한 것이다. 

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [46]:
# 단어 빈도만 가지고 임베딩을 하는 것은 문제가 있다. 이것이 얼마만큼 중요한 단어인지 설명할 수 있어야 한다.
# TF-IDF(Term Frequency - Inverse Document Frequency) 단어빈도*역문서빈도의 약자이다. 
# 즉 문서에서 각 "단어의 중요도"를 나타낸 행렬이다. 
# 구하는 목적은 각 문서에서 중요단어가 무엇인지 궁금하기에 사용한다. 
# 키워드로 만들고, 더 나아가 토픽모델링으로 확장해서 사용하기도 한다. 
# 여기서 단어의 중요하다는 의미는 단어가 자주 나올때 중요하다고 인식하는 것이다.
# 문제는 단순히 많이 등장했다고 해서 중요도가 높지는 않을 것이다. 
# 모든 문장에는 관사가 존재한다. 관사가 중요하다고 말하지는 않을 것이다.
# df는 단어의 빈도수이다.위에 먹고는 2번 나왔고 바나나는 3번 나왔다. df가 높다는 것은 많은 단어라는 뜻이다. 
# 즉 df가 작아져아지 중요도가 높다고 판단한 것이다. 
# 로그를 취하는 이유 : 로그를 취하지 않았을 때 문서의 갯수가 커질수록 idf값이 기하급수적으로 커진다. 
# 로그를 안쓰면 idf값이 마구마구 커진다. 1백만은 log 6 이다.  로그를 안쓰면 1백만을 그대로 써야한다. 그리고 log1 = 0 이다. 

# 요소연산(element predict) : 요소끼리 곱셈을 하는 것이다. 
# 행렬곱셈(matrix multifly) : 행렬의 전체곱셈, 위치하는 방식이다. 우리가 일반적으로 쓰는 곱셈이다.
# 문서빈도(Term Frequency)는 위에 '먹고'라는 단어가 몇개에 문서에서 등장했냐는 것이다. 2개의 문서에서 등장했을 것이다.
# 바나나도 2이다 문서에 바나나가 나왔는지만 보면 2이다. 문서 내 갯수는 상관없다.


In [47]:
# idf
from sklearn.feature_extraction.text import TfidfVectorizer #Tfidf 벡터라이저이다. 

In [48]:
result = []
for j in range(len(vocab)): 
    t = vocab[j] # 각각의 단어를 t에 대입했다. 
    result.append(idf(t)) # idf값으로 계산하여 결과값에 대입한다.

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"]) #행의 이름을  각 단어들로 했고 열값을 IDF라고 정의했다. 
idf_ # idf행렬이 출력되었다. 먹고, 바나나는 흔한 단어라는 소리이다. 

,IDF
먹고,0.287682
좋아요,0.693147
싶은,0.287682
과일이,0.693147
길고,0.693147
노란,0.693147
사과,0.693147
바나나,0.287682
저는,0.693147


In [49]:
tf_ # tf도 만들어졌다.

,먹고,좋아요,싶은,과일이,길고,노란,사과,바나나,저는
0,1,0,1,0,0,0,1,0,0
1,1,0,1,0,0,0,0,1,0
2,0,0,0,0,1,1,0,2,0
3,0,1,0,1,0,0,0,0,1


In [96]:
# 이제 이 두 개를 곱하면 된다. 
# 이제 행렬곱으로 tf와 idf를 곱하면 되는 것이다. 
result = []
for i in range(N): # n개의 문서를 돌았다.
  result.append([]) # 먼저 문서에 리스트를 추가했다.
  d = docs[i] # 문서를 d로 정으했다.
  for j in range(len(vocab)): # 문자의 길이만큼 정의했다.
    t = vocab[j] # 문자를 하나하나 출력해서 t에 대입했따.
    result[-1].append(tfidf(t,d)) # 결과값에 tfidf로 계산해서 새로생긴 리스트에 대입한다. 

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,먹고,좋아요,싶은,과일이,길고,노란,사과,바나나,저는
0,0.287682,0.000000,0.287682,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000
1,0.287682,0.000000,0.287682,0.000000,0.000000,0.000000,0.000000,0.287682,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000
3,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147


In [51]:
# 0. 데이터 전처리를 잘 시행해보자. 불용어 제거 등의 절차를 해야 한다. 
# 1. 각 문서(영화)에서 중요 단어 추출
# 2. 코사인유사도 기반 가장 유사한 문서(영화) 검색
# https://scholar.google.com/scholar?hl=ko&as_sdt=0%2C5&q=tf+idf+cosine+similarity&oq=tfidf+cosin 
# 학술논문이다.2023년 논문들이다. 
# 어느 것에 대한 가중치를 부여해서 tf-idf를 사용할 수도 있다. 
# 오후에 해보도록 하자. 
# 영어전처리는nltpl을 사용한다.

In [54]:
#nltk 패키지 이용가이드
!pip install tensorflow # 사용하기 위해서는 텐서플로를 설치해야 한다. 

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5

In [56]:
pip install gensim # 젠심을 설치하자. 

     ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
     ---------------------------------------- 67.1/67.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20448 sha256=821c32cf529edd6fdf1f0f1165f93e12e11eb4fe07f6d4fb54157ee1406d1a27
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\69\f5\e5\18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3522 sha256=90e66a285e4a38decbdfb4c098d75d094dcf47971a93729eb0dfcdae4f78ea18
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\9d\ff\2f\afe4cd56f47de147407705626517d68bea0f3b74eb1fb168e6
Successfully built fst-pso miniful
Note: you may need to restart the kernel to use up

In [57]:
pip install nltk # nltk를 최종적으로 설치했다. 

Note: you may need to restart the kernel to use updated packages.


In [61]:
from nltk.tokenize import word_tokenize # 영어로 된 글자를 토큰화시킨다. 
from nltk.tokenize import WordPunctTokenizer # 위랑 비슷하다. 다만 점을 분리하는 방식이 다르다. 
from tensorflow.keras.preprocessing.text import text_to_word_sequence # 토큰화를 텐서플로가 해준다. 단어를 전부 소문자로 바꾼다. 
# 버젼도 맞춰야 하는 과정들이 필요할 수 있다. 꽤 번거롭다. 

In [59]:
# natural language tokenizetion 
# 토큰화를 하기 위한 셋이다. 
# 토큰 : 단어(문장)을 말한다. 단어단위로 나눌 수 있고 문장단위로 나눌 수도 있다.
# 심지어는 문단을 토큰으로 볼 수도 있다. 심하면 문자도 토큰이 될 수 있다.(단어, 문장, 문단, 문자)  
# 즉 토큰은 자연어처리 작업을 수행하는 기본 단위를 말한다. 
# 토큰화 : 주어진 코퍼스를 토큰 단위로 나누는 작업을 말한다. 
# 자연어 -> 토큰화 -> 세부작업을 진행하는 식이다. 

In [60]:
print('단어 토큰화1 :',word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'C:\\Users\\user\\anaconda3\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [62]:
import nltk
nltk.download('punkt')
# 룩업애러는 이렇게 하면 된다. 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [63]:
print('단어 토큰화1 :',word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))
# 잘 작동하였다. 룩업애러에서 다운로드 하라고 하는 것은 다 다운받으면 된다. 


단어 토큰화1 : ['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


In [64]:
print('단어 토큰화2 :',WordPunctTokenizer().tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))
# 위랑 처리하는 방식이 다르다. 영어에도 토큰화 도구가 있지만 많이 다르다. 

단어 토큰화2 : ['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


In [65]:
print('단어 토큰화3 :',text_to_word_sequence("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))
# 케라스에서 제공하는 도구이다.
# 모든 문자를 소문자로 바꾼다. 특이하게 콤마가 없어졌다. 마침표도 제거해버린다. 즉 구두점(fuctuation)을 전부 날린다. 
# don't가 그대로 보존된다. jone's도 보존되었다. 
# 이걸 단어토큰화라고 한다. 물론 문장토큰화도 존재한다. 

단어 토큰화3 : ["don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'mr', "jone's", 'orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


In [66]:
from nltk.tokenize import sent_tokenize # 문장을 문단으로 토큰화하였다. 자료형 리스트로 담는다. 

text = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to make sure no one was near."
print('문장 토큰화1 :',sent_tokenize(text))
# 문장별로 토큰화를 하였다. 

문장 토큰화1 : ['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to make sure no one was near.']


In [67]:
text = "I am actively looking for Ph.D. students. and you are a Ph.D student."
print('문장 토큰화2 :',sent_tokenize(text))
# 재밌는건 단순하게 마침표로 문장을 인식하지 않는다. 마침표만으로 문장의 끝으로 안본다. 성능이 괜찮다. 
# 이와같은 것들이 존재한다.

문장 토큰화2 : ['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


In [68]:
# 오후에는 전처리를 좀더 소개하고 어제 사용한 영화데이터로 전처리를 거칠 것이다.
# 코사인 유사도를 구해서 오버뷰를 구해서 가장 유사한 영화를 찾아주는 시스템을 만들 것이다. 
# 직접 구축해볼 것이다. 
# 자연어는 범주가 매우 넓고 이번것은 극히 일부분이다. 
# 앞시간에 영어의 토크나이즈를 확인했다. 복습을 해보자.

In [69]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# pos는 파트오브 스피치다. 
# 이런 식으로 형태소 분석이 이루어졌다. 

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


In [71]:
print('꼬꼬마 형태소 분석 :',kkma.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('꼬꼬마 품사 태깅 :',kkma.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('꼬꼬마 명사 추출 :',kkma.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# 자연어 처리를 할 때는 토큰화 작업을 먼저 한다. 
# 토큰화작업 이후에는 정제를 해야한다. 불필요한 단어를 제거하는 절차이다. 불용어라고 한다.
# 불용어는 길이가 짧은 단어나 등장 빈도수가 낮은 단어이다. 
# 영화 오버뷰에는 단어가 너무 많을 것이다. 그래서 일일히 출력하면 다 못나온다. 
# 불용어 제거는 등장빈도수가 낮은 단어를 선택해야 하는데 이 수치를 얼마로 할 것인지가 애매하다. 
# 상황에 맞는 기준을 정해서 만들어야 한다. 
# 만약 5000편의 영화를 가지고 연관분석을 한다면 전체 영화에 대해서 열의 합계를 모은다. 
# 열단위로 합계수치가 5 미만이면 제거. 이런 식으로 진행한다. 

꼬꼬마 형태소 분석 : ['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가보', '아요']
꼬꼬마 품사 태깅 : [('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가보', 'VV'), ('아요', 'EFN')]
꼬꼬마 명사 추출 : ['코딩', '당신', '연휴', '여행']


In [72]:
import re
text = "I was wondering if anyone out there could enlighten me on this car."

# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제
shortword = re.compile(r'\W*\b\w{1,2}\b')# 단어나 문자나 숫자가 아닌것과 매치, 공백문자, 문자숫자, 1글자 이상, 2글자 이하에 부합하는 경우,공백문자.
print(shortword.sub('', text)) # 숏워드패턴의 글자는 전부 지워라.
# 정규표현식을 이용해서불필요한 단어를 제거하는 작업을 한 것이다. 

 was wondering anyone out there could enlighten this car.


In [73]:
# 불용어 제거는 불용어 사전을 이용해서 제거할 수도 있다. 
# 단어에서도 중요한 부분이 어간이다. 부가 되는 것을 접사라고 한다.
# 형태소는 어간(stem, 단어의 중요의미)과 접사(부가의미)로 구분된다. 
# 예를 들어서 영어에 dogs가 있다고 해보자.
# dogs => dog + s 로 정의할 수 있다. 
# dog부터는 분리할 수 없다. 
# 필요하면 쓰고 필요하면 쓰지 않아도 된다. 

In [74]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('표제어 추출 전 :',words)
print('표제어 추출 후 :',[lemmatizer.lemmatize(word) for word in words])

표제어 추출 전 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'C:\\Users\\user\\anaconda3\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [75]:
import nltk # 애러 메세지가 뜨면 이걸 다운로드 하라고 나올 것이다.
nltk.download('wordnet')# 그리고 다시 실행해보자.

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [76]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()# 레머타이저란 단어들 중 are, is와 같은 am같은 be(표제어)를 말한다. 

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('표제어 추출 전 :',words)
print('표제어 추출 후 :',[lemmatizer.lemmatize(word) for word in words])
# 리브가 라이프로 바꾼다. 

표제어 추출 전 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
표제어 추출 후 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


In [77]:
lemmatizer.lemmatize('is','v')

'be'

In [78]:
lemmatizer.lemmatize('are','v')

'be'

In [79]:
lemmatizer.lemmatize('watched','v')

'watch'

In [80]:
lemmatizer.lemmatize('watching','v')
# 단어정리를 할 때 매우 유용해보인다. 

'watch'

In [81]:
# 어간을 추출하는 작업을 해보자.(stemming)
# 성능이 좋지만은 않다. 영어권의 학자들 중 쿼터, 레케스터가 만든 스팸어같은 것이다.
# 약간씩 성능이 다르다. 

In [87]:
from nltk.stem import PorterStemmer # 포토스테머를 통해 스태밍을 해보자. 
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()

In [85]:
words = ['formalize', 'allowance', 'electricical'] # 어간을 추출할 대상이다.
print('어간 추출 후 :',[stemmer.stem(word) for word in words])# 어간 추출 결과이다. 
# 이걸 만든 사람의 사이트이다. https://tartarus.org/martin/PorterStemmer/

어간 추출 후 : ['formal', 'allow', 'electric']


In [88]:
print('어간 추출 후 :',[lancaster_stemmer.stem(word) for word in words])# 레케스터 스테머는 뭔가 나사가 빠져있다. 

어간 추출 후 : ['form', 'allow', 'elect']


In [90]:
from nltk.corpus import stopwords # 불용어 제거작업을 해보자.
# 

In [91]:
stopwords.words('english')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'C:\\Users\\user\\anaconda3\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\user\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [92]:
import nltk #이것도 다운로드 받아야 한다.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [93]:
stopwords.words('english') # 불용어 사전이다. 
# 200여개의 단어가 있다. 

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [94]:
example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) # 불용어사전을 할당했다. 

word_tokens = word_tokenize(example) # 단어를 토큰화해서 담았따. 

result = []
for word in word_tokens: 
    if word not in stop_words: 
        result.append(word) # 단어가 만약 불용어사전에 없으면 담는다. 

print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)
# 불용어를 제거하면서 is not an 이 빠졌다. 

불용어 제거 전 : ['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
불용어 제거 후 : ['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


In [95]:
# 만약에 불용어를 추가하고 싶다면 stop_words에 append로 추가하자.
# 한국어 불용어를 제거해보자. 
example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
stop_words = "를 아무렇게나 구 우려 고 안 돼 같은 게 구울 때 는" # 불용어를 설정했다.

stop_words = set(stop_words.split(' ')) # 불용어를 나눠서 세트에 할당했따. 
word_tokens = okt.morphs(example) 

result = [word for word in word_tokens if not word in stop_words]

print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)

불용어 제거 전 : ['고기', '를', '아무렇게나', '구', '우려', '고', '하면', '안', '돼', '.', '고기', '라고', '다', '같은', '게', '아니거든', '.', '예컨대', '삼겹살', '을', '구울', '때', '는', '중요한', '게', '있지', '.']
불용어 제거 후 : ['고기', '하면', '.', '고기', '라고', '다', '아니거든', '.', '예컨대', '삼겹살', '을', '중요한', '있지', '.']
